In [1]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.update()
# constraints = model.getConstrs()

# constraints[0].CBasis = - 1
# constraints[1].CBasis = - 1

# # x_j.VBasis = [0, 0]
# variables = model.getVars()
# variables[0].VBasis = 0
# variables[1].VBasis = 0


# model.optimize()

Set parameter TSPort to value 61954
Set parameter TokenServer to value "10.0.1.1"
Set parameter Method to value 0


# Using .sol

In [2]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.optimize()


model.write('test.sol')

# Save model
model.write('test.mps')
model.write('test.bas')

print(model.x)

Set parameter Method to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xf17fe891
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   2.000000e+00      0s
       2    1.3333333e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.333333333e+00
[0.6666666666666666, 0.6666666666666667]


In [3]:
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3333333e+00   2.500000e-01   7.499993e+05      0s
       1    1.1111111e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.111111111e+00


In [4]:
model = gp.read('test.mps')
x_j = model.getVars()
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.update()
model.read('test.bas')
# model.optimize()
# print(model.x)

Read MPS format model from file test.mps
Reading time = 0.00 seconds
: 2 rows, 2 columns, 4 nonzeros
Read basis from file test.bas


In [5]:
print(model.getVars()[0].VBasis)
print(model.getConstrs()[2].CBasis)

0
0


# Master pb

In [4]:
import gurobipy as gp
path = 'GMM_quad_v1-100/output/'

master_pb = gp.read(path + 'master_pb.mps')
master_pb.read(path +'master_pb.bas')

Read MPS format model from file GMM_quad_v1-100/output/master_pb.mps
Reading time = 1.45 seconds
GMM_pb: 347052 rows, 25998 columns, 5682491 nonzeros
Read basis from file GMM_quad_v1-100/output/master_pb.bas


In [5]:
basic = [var for var in master_pb.getVars() if var.VBasis == 0]
non_basic = [var for var in master_pb.getVars() if var.VBasis == -1]
print(len(basic), len(non_basic))

slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == 0]
non_slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == -1]
print(len(slack), len(non_slack))

17054 8944
329998 17054


In [ ]:
# master_pb.optimize()

# basic = [var for var in master_pb.getVars() if var.VBasis == 0]
# non_basic = [var for var in master_pb.getVars() if var.VBasis == -1]
# print(len(basic), len(non_basic))

# slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == 0]
# non_slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == -1]
# print(len(slack), len(non_slack))


In [29]:
A = master_pb.getA()[:,5:]

# transform into scipy sparse matrix
from scipy.sparse import csr_matrix
A = csr_matrix(A)

from scipy.sparse.linalg import svds

# Perform Singular Value Decomposition (SVD) and determine rank
U, s, Vh = svds(A, k=min(A.shape)-1)  # k=shape-1 ensures full SVD
rank = (s > 1e-10).sum()  # Count significant singular values
print(f"The rank of the sparse matrix is: {rank}")

: 

: 

: 

Dual solution

In [38]:
import numpy as np
dual_solutions_master_pb = np.load('/scratch/ed2189/JMP/GMM_quad_v1-100/output/dual_solution_master_pb.npy')

# Pricing

In [1]:
import numpy as np
import gurobipy as gp

In [9]:
num_items = 50

values_j = np.random.randint(0,100,num_items)*1.0

values_j *= np.random.choice([-1e10,1], num_items, p=[.2, .8])



density = .1
Q_j_j = np.random.choice([0,1], (num_items,num_items), p=[1-density, density])


In [12]:
model = gp.Model()

x_j = model.addMVar(num_items, lb=0, ub=1, vtype=gp.GRB.BINARY)

# Objective
model.setObjective(values_j @ x_j + x_j @ Q_j_j @ x_j, gp.GRB.MAXIMIZE)

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
TSPort  61954

Optimize a model with 0 rows, 50 columns and 0 nonzeros
Model fingerprint: 0x9d67683c
Model has 223 quadratic objective terms
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 8e+11]
  QObjective range [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 1909.0000000
Presolve removed 0 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simpl